In [5]:
from os import path

In [134]:
dummyfile = "vg~vg-20111003-10039711"
dummyfile = "aftenposten_01"
dummyfile = "ap_20081210-1252467"
dummyfile = "ap_20051201-18064"
# dummyfile = "nou~nou_003_a"
ext = ".ann"
file_path = path.join("../../annotations", dummyfile + ext)

data = None
with open(file_path, "r", encoding="utf-8") as f:
    data = f.readlines()
    

In [140]:

def get_references(ann_data):
    """ convert the raw ann data to referential objects

    Args:
        ann_data (List[str]): raw data from .ann file
    Returns:
        Tuple[Markable, List[MarkableReference]]:
            a dictionary of markable items and a list of coreferences
    """
    markables = {}
    corefs = []

    for line in ann_data:
        try:
            _id, ref_data, text = line.split("\t")
        except ValueError as err:
            print(f"Failed to read {line}: {err} -- Ignoring and continuing")
            continue
        if "R" in _id:
            _type, arg1, arg2 = ref_data.split()
            # ignore bridging
            if _type == "Coref":
                m1 = arg1.split(":")[-1]
                m2 = arg2.split(":")[-1]
                corefs.append((m1, m2))

        else:
            ref_data = ref_data.split()
            markables[_id] = {
                "start": ref_data[1],  # index 0 is simply "Markable"
                "end": ref_data[-1],
                "text": text
            }

    return markables, corefs

markables, corefs = get_references(data)
markables

{'T1': {'start': '0', 'end': '7', 'text': 'Sofia :\n'},
 'T2': {'start': '8', 'end': '20', 'text': 'Allan Gaarde\n'},
 'T3': {'start': '36', 'end': '50', 'text': 'tiden i Viking\n'},
 'T4': {'start': '44', 'end': '50', 'text': 'Viking\n'},
 'T5': {'start': '57', 'end': '66', 'text': 'sommerens\n'},
 'T6': {'start': '57',
  'end': '87',
  'text': 'sommerens overgang fra Aalborg\n'},
 'T7': {'start': '80', 'end': '87', 'text': 'Aalborg\n'},
 'T8': {'start': '94', 'end': '97', 'text': 'han\n'},
 'T9': {'start': '128', 'end': '139', 'text': 'resultatene\n'},
 'T10': {'start': '144', 'end': '147', 'text': 'Jeg\n'},
 'T11': {'start': '157', 'end': '160', 'text': 'det\n'},
 'T12': {'start': '172', 'end': '177', 'text': 'banen\n'},
 'T13': {'start': '215', 'end': '218', 'text': 'Jeg\n'},
 'T14': {'start': '227', 'end': '233', 'text': 'Viking\n'},
 'T15': {'start': '246', 'end': '249', 'text': 'noe\n'},
 'T16': {'start': '269', 'end': '272', 'text': 'meg\n'},
 'T17': {'start': '285', 'end': '28

In [138]:
list(markables.items())[15:20]

[('T16', {'start': '269', 'end': '272', 'text': 'meg\n'}),
 ('T17', {'start': '285', 'end': '287', 'text': 'vi\n'}),
 ('T18', {'start': '297', 'end': '305', 'text': 'medaljer\n'}),
 ('T19', {'start': '308', 'end': '314', 'text': 'serien\n'}),
 ('T20', {'start': '331', 'end': '347', 'text': 'for mye stang ut\n'})]

In [79]:
def arg_to_markable(arg: str) -> str:
    return arg.split(":")[-1]

# second pass to update references with markable data
corefs = []
for line in data:
    if len(line.split()) != 4:
        continue
    _id, _type, arg1, arg2 = line.split(maxsplit=4)
    
    if _type != "Brdiging":
        source = arg_to_markable(arg1)
        to = arg_to_markable(arg2)
        corefs.append((source, to))
corefs

[('T8', 'T2'),
 ('T13', 'T10'),
 ('T14', 'T4'),
 ('T16', 'T13'),
 ('T28', 'T24'),
 ('T29', 'T3'),
 ('T30', 'T17'),
 ('T35', 'T32'),
 ('T37', 'T35'),
 ('T41', 'T37'),
 ('T46', 'T43'),
 ('T44', 'T41'),
 ('T49', 'T44'),
 ('T54', 'T51'),
 ('T55', 'T54'),
 ('T58', 'T55'),
 ('T63', 'T62'),
 ('T68', 'T66'),
 ('T69', 'T61'),
 ('T72', 'T69'),
 ('T74', 'T25'),
 ('T10', 'T8'),
 ('T24', 'T16'),
 ('T31', 'T28'),
 ('T32', 'T30'),
 ('T43', 'T31'),
 ('T51', 'T46'),
 ('T62', 'T58'),
 ('T73', 'T63'),
 ('T57', 'T12')]

In [80]:
# create a mapping from markable -> all mentions
markable_coref_map = {}
for mark in markables.keys():
    mark_refs = []
    for coref in corefs:
        if mark in coref:
            mark_refs.append(coref)
    if len(mark_refs) > 0:
        markable_coref_map[mark] = mark_refs
        
markable_coref_map


{'T2': [('T8', 'T2')],
 'T3': [('T29', 'T3')],
 'T4': [('T14', 'T4')],
 'T8': [('T8', 'T2'), ('T10', 'T8')],
 'T10': [('T13', 'T10'), ('T10', 'T8')],
 'T12': [('T57', 'T12')],
 'T13': [('T13', 'T10'), ('T16', 'T13')],
 'T14': [('T14', 'T4')],
 'T16': [('T16', 'T13'), ('T24', 'T16')],
 'T17': [('T30', 'T17')],
 'T24': [('T28', 'T24'), ('T24', 'T16')],
 'T25': [('T74', 'T25')],
 'T28': [('T28', 'T24'), ('T31', 'T28')],
 'T29': [('T29', 'T3')],
 'T30': [('T30', 'T17'), ('T32', 'T30')],
 'T31': [('T31', 'T28'), ('T43', 'T31')],
 'T32': [('T35', 'T32'), ('T32', 'T30')],
 'T35': [('T35', 'T32'), ('T37', 'T35')],
 'T37': [('T37', 'T35'), ('T41', 'T37')],
 'T41': [('T41', 'T37'), ('T44', 'T41')],
 'T43': [('T46', 'T43'), ('T43', 'T31')],
 'T44': [('T44', 'T41'), ('T49', 'T44')],
 'T46': [('T46', 'T43'), ('T51', 'T46')],
 'T49': [('T49', 'T44')],
 'T51': [('T54', 'T51'), ('T51', 'T46')],
 'T54': [('T54', 'T51'), ('T55', 'T54')],
 'T55': [('T55', 'T54'), ('T58', 'T55')],
 'T57': [('T57', 'T12')]

In [110]:
# we cannot guarantee that the ordering is proper,
# thus a single pass through the markables will never be sufficient
entity_id = 0
grouped_map = {}

def visit(coref, added, visited):
    if coref in visited:
        return
    visited.add(coref)
    for markable in coref:
        added.append(coref)
        # add all tuples for this value...
        child_values = markable_coref_map[markable]
        for cv in child_values:
            if cv not in visited:
                #print("not visited:", cv)
                return visit(cv, added, visited)
    return list(set(added))
        
visited = set()
grouped = {}
entity_id = 0
for corefs in markable_coref_map.values():
    for coref in corefs:
        # print(coref)
        tmp_cluster = visit(coref, [], visited)
        if tmp_cluster:
            grouped[entity_id] = tmp_cluster
            entity_id += 1
grouped

{0: [('T10', 'T8'),
  ('T28', 'T24'),
  ('T58', 'T55'),
  ('T8', 'T2'),
  ('T31', 'T28'),
  ('T63', 'T62'),
  ('T24', 'T16'),
  ('T62', 'T58'),
  ('T46', 'T43'),
  ('T16', 'T13'),
  ('T51', 'T46'),
  ('T54', 'T51'),
  ('T43', 'T31'),
  ('T13', 'T10'),
  ('T73', 'T63'),
  ('T55', 'T54')],
 1: [('T29', 'T3')],
 2: [('T14', 'T4')],
 3: [('T57', 'T12')],
 4: [('T35', 'T32'),
  ('T44', 'T41'),
  ('T41', 'T37'),
  ('T49', 'T44'),
  ('T37', 'T35'),
  ('T30', 'T17'),
  ('T32', 'T30')],
 5: [('T74', 'T25')],
 6: [('T72', 'T69'), ('T69', 'T61')],
 7: [('T68', 'T66')]}

In [111]:
# convert to indices
cluster_idxs = dict()

def markable_to_idx(mark):
    obj = markables[mark]
    return (int(obj["start"]), int(obj["end"]))

def markable_to_idxs(marks):
    start = markable_to_idx(marks[0])
    end = markable_to_idx(marks[1])
    return start, end

for cluster_id, cluster_markables in grouped.items():
    cluster_idxs[cluster_id] = [markable_to_idxs(marks) for marks in cluster_markables]

In [112]:
cluster_idxs

{0: [((144, 147), (94, 97)),
  ((472, 475), (388, 395)),
  ((1032, 1045), (971, 974)),
  ((94, 97), (8, 20)),
  ((532, 535), (472, 475)),
  ((1089, 1092), (1082, 1085)),
  ((388, 395), (269, 272)),
  ((1082, 1085), (1032, 1045)),
  ((783, 789), (732, 738)),
  ((269, 272), (215, 218)),
  ((892, 895), (783, 789)),
  ((962, 968), (892, 895)),
  ((732, 738), (532, 535)),
  ((215, 218), (144, 147)),
  ((1228, 1234), (1089, 1092)),
  ((971, 974), (962, 968))],
 1: [((506, 527), (36, 50))],
 2: [((227, 233), (44, 50))],
 3: [((1013, 1018), (172, 177))],
 4: [((607, 609), (542, 544)),
  ((744, 750), (694, 696)),
  ((694, 696), (660, 663)),
  ((859, 861), (744, 750)),
  ((660, 663), (607, 609)),
  ((514, 520), (285, 287)),
  ((542, 544), (514, 520))],
 5: [((1239, 1253), (400, 414))],
 6: [((1199, 1201), (1164, 1166)), ((1164, 1166), (1082, 1100))],
 7: [((1156, 1159), (1119, 1150))]}

In [123]:
# note that we above have the character indices for the references, we want to convert this to word indices
# to support the format used for multiple models.

# fetch the full-text:
ext = ".txt"  #  .ann
file_path = path.join("../../annotations", dummyfile + ext)

data = None
with open(file_path, "r", encoding="utf-8") as f:
    data = f.readlines()

data[:1000]

['Sofia :\n',
 'Allan Gaarde er fornøyd med tiden i Viking etter sommerens overgang fra Aalborg .\n',
 'Men han er bare " semi-tilfreds " med resultatene .\n',
 '- Jeg har fått det bra til på banen , men er likevel ikke helt fornøyd .\n',
 'Jeg kom til Viking for å vinne noe , og det irriterer meg fortsatt at vi ikke tok medaljer i serien .\n',
 'Det har blitt for mye stang ut i høst , håper det snur neste år , sier dansken til aftenbladet.no .\n',
 '- Gode nok til å gå videre |\n',
 'Høstens formsvikt gjør at han bare er " semi-tilfreds " med tiden i Viking til nå .\n',
 '- Jeg mener vi var gode nok til å gå videre fra gruppespillet i Uefa-cupen .\n',
 'Vi burde slått Slavia Praha , og kunne lett fått med oss poeng her fra Sofia .\n',
 'Synd at vi ikke klart å ta flere poeng , sier Gaarde .\n',
 'At Viking ikke kom til Royal League , tar Gaarde ikke så tungt .\n',
 '- Mer økonomi enn sport |\n',
 '- Det betyr ikke så mye at vi ikke kom med i Royal League .\n',
 'Jeg føler at den turne

In [124]:
data = "".join(data)
data = data.replace("\n\n", " <N> ")
data[:1000]

'Sofia :\nAllan Gaarde er fornøyd med tiden i Viking etter sommerens overgang fra Aalborg .\nMen han er bare " semi-tilfreds " med resultatene .\n- Jeg har fått det bra til på banen , men er likevel ikke helt fornøyd .\nJeg kom til Viking for å vinne noe , og det irriterer meg fortsatt at vi ikke tok medaljer i serien .\nDet har blitt for mye stang ut i høst , håper det snur neste år , sier dansken til aftenbladet.no .\n- Gode nok til å gå videre |\nHøstens formsvikt gjør at han bare er " semi-tilfreds " med tiden i Viking til nå .\n- Jeg mener vi var gode nok til å gå videre fra gruppespillet i Uefa-cupen .\nVi burde slått Slavia Praha , og kunne lett fått med oss poeng her fra Sofia .\nSynd at vi ikke klart å ta flere poeng , sier Gaarde .\nAt Viking ikke kom til Royal League , tar Gaarde ikke så tungt .\n- Mer økonomi enn sport |\n- Det betyr ikke så mye at vi ikke kom med i Royal League .\nJeg føler at den turneringen betyr mer økonomisk enn sportslig , sier Gaarde .\nHan har funne

In [125]:
" .\nDet er ovnsbakt ".split()

['.', 'Det', 'er', 'ovnsbakt']

In [126]:
# we need to tokenize on spaces to convert the characters to words...
tokens = data.split()

In [127]:
char_map = {}
char_idx = 0
for i, tok in enumerate(tokens):
    # char_map[char_idx] = (tok, i)
    char_map[char_idx] = {
        "token": tok,
        "word_index": i
    }
    incr = len(tok) + 1  # +1 to account for subsequent spaces
    if tok == "<N>":
        # only increment by 1
        incr = 1
    char_idx += incr  
    

In [128]:
list(char_map.items())

[(0, {'token': 'Sofia', 'word_index': 0}),
 (6, {'token': ':', 'word_index': 1}),
 (8, {'token': 'Allan', 'word_index': 2}),
 (14, {'token': 'Gaarde', 'word_index': 3}),
 (21, {'token': 'er', 'word_index': 4}),
 (24, {'token': 'fornøyd', 'word_index': 5}),
 (32, {'token': 'med', 'word_index': 6}),
 (36, {'token': 'tiden', 'word_index': 7}),
 (42, {'token': 'i', 'word_index': 8}),
 (44, {'token': 'Viking', 'word_index': 9}),
 (51, {'token': 'etter', 'word_index': 10}),
 (57, {'token': 'sommerens', 'word_index': 11}),
 (67, {'token': 'overgang', 'word_index': 12}),
 (76, {'token': 'fra', 'word_index': 13}),
 (80, {'token': 'Aalborg', 'word_index': 14}),
 (88, {'token': '.', 'word_index': 15}),
 (90, {'token': 'Men', 'word_index': 16}),
 (94, {'token': 'han', 'word_index': 17}),
 (98, {'token': 'er', 'word_index': 18}),
 (101, {'token': 'bare', 'word_index': 19}),
 (106, {'token': '"', 'word_index': 20}),
 (108, {'token': 'semi-tilfreds', 'word_index': 21}),
 (122, {'token': '"', 'word_in

In [129]:
from typing import Tuple

IndexRange = Tuple[int, int]

def char_to_word_index(chars: IndexRange) -> IndexRange:
    start, end = chars
    index_range = list(range(start, end + 1))
    # map it to valid keys
    index_range = [_i for _i in range(start, end + 1) if _i in char_map.keys()]
    span = [char_map[_j] for _j in index_range]

    start_idx = span[0]["word_index"]
    end_idx = span[-1]["word_index"]

    return start_idx, end_idx
    
test_span = (135, 159)
char_to_word_index(test_span)



(25, 30)

In [130]:
word_indice_clusters = []

for c_idx in cluster_idxs.values():
    tmp_cluster = set()
    for start, end in c_idx:
        tmp_cluster.add(char_to_word_index(start))
        tmp_cluster.add(char_to_word_index(end))
    tmp_cluster = list(tmp_cluster)
    word_indice_clusters.append(tmp_cluster)
        
        # print(tokens[w_start:w_end + 1])
        # print(tokens[w2_start:w2_end + 1])

In [131]:
word_indice_clusters

[[(17, 17),
  (98, 98),
  (219, 220),
  (27, 27),
  (155, 155),
  (43, 43),
  (206, 206),
  (192, 192),
  (262, 262),
  (2, 3),
  (112, 112),
  (228, 228),
  (230, 230),
  (82, 82),
  (55, 55),
  (166, 166),
  (204, 204)],
 [(7, 9), (105, 109)],
 [(9, 9), (46, 46)],
 [(34, 34), (214, 214)],
 [(114, 114),
  (158, 158),
  (107, 107),
  (138, 138),
  (58, 58),
  (146, 146),
  (127, 127),
  (184, 184)],
 [(264, 264), (84, 84)],
 [(256, 256), (228, 231), (247, 247)],
 [(245, 245), (236, 242)]]

In [132]:
for wc in word_indice_clusters:
    print(wc)
    for mention in wc:
        start, end = mention
        print(tokens[start:end + 1])
    print("__")

[(17, 17), (98, 98), (219, 220), (27, 27), (155, 155), (43, 43), (206, 206), (192, 192), (262, 262), (2, 3), (112, 112), (228, 228), (230, 230), (82, 82), (55, 55), (166, 166), (204, 204)]
['han']
['han']
['meg', 'personlig']
['Jeg']
['Gaarde']
['Jeg']
['Han']
['Jeg']
['Gaarde']
['Allan', 'Gaarde']
['Jeg']
['Jeg']
['min']
['dansken']
['meg']
['Gaarde']
['Gaarde']
__
[(7, 9), (105, 109)]
['tiden', 'i', 'Viking']
['tiden', 'i', 'Viking', 'til', 'nå']
__
[(9, 9), (46, 46)]
['Viking']
['Viking']
__
[(34, 34), (214, 214)]
['banen']
['banen']
__
[(114, 114), (158, 158), (107, 107), (138, 138), (58, 58), (146, 146), (127, 127), (184, 184)]
['vi']
['Viking']
['Viking']
['oss']
['vi']
['vi']
['Vi']
['vi']
__
[(264, 264), (84, 84)]
['aftenbladet.no']
['aftenbladet.no']
__
[(256, 256), (228, 231), (247, 247)]
['vi']
['Jeg', 'og', 'min', 'samboer']
['vi']
__
[(245, 245), (236, 242)]
['det']
['et', 'lite', 'eventyr', 'i', 'et', 'annet', 'land']
__
